Dataset : https://www.kaggle.com/datasets/kabilan45/online-retail-ii-dataset

In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default = "plotly_white"
from google.colab import drive

In [2]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
data = pd.read_excel("/content/gdrive/MyDrive/Reference/Data Science/Fundamental/Data Science Project/Personal/RFM/online_retail_II.xlsx")

In [4]:
data.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [5]:
data['TransactionAmount'] = data['Quantity']* data['Price']

In [6]:
data.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TransactionAmount
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0


**Calculating RFM Values**
I’ll now calculate the Recency, Frequency, and Monetary values of the customers to move further:

In [7]:
import pandas as pd
from datetime import datetime

data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
data['Recency'] = (datetime.now() - data['InvoiceDate']).dt.days
frequency_data = data.groupby('Customer ID')['Invoice'].count().reset_index()
frequency_data.rename(columns={'Invoice':'Frequency'},inplace=True)
data = data.merge(frequency_data,on='Customer ID',how='left')
monetary_data = data.groupby('Customer ID')['TransactionAmount'].sum().reset_index()
monetary_data.rename(columns={'TransactionAmount':'MonetaryValue'},inplace=True)
data = data.merge(monetary_data,on='Customer ID',how='left')
data = data.drop_duplicates(subset=['Customer ID'])

In [8]:
data.tail()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TransactionAmount,Recency,Frequency,MonetaryValue
521212,537817,22837,HOT WATER BOTTLE BABUSHKA,4,2010-12-08 13:53:00,4.65,18269.0,United Kingdom,18.6,5028,7.0,168.60
522042,537833,51008,AFGHAN SLIPPER SOCK PAIR,200,2010-12-08 14:53:00,2.95,13270.0,United Kingdom,590.0,5028,1.0,590.00
522621,537885,48185,DOORMAT FAIRY CAKE,2,2010-12-09 09:34:00,7.95,12942.0,United Kingdom,15.9,5027,16.0,258.75
522900,537963,37446,MINI CAKE STAND WITH HANGING CAKES,32,2010-12-09 11:30:00,1.25,13369.0,United Kingdom,40.0,5027,14.0,308.28
524732,538145,22224,WHITE LOVEBIRD LANTERN,6,2010-12-09 16:08:00,2.95,15211.0,United Kingdom,17.7,5027,23.0,383.07


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4384 entries, 0 to 524732
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Invoice            4384 non-null   object        
 1   StockCode          4384 non-null   object        
 2   Description        4384 non-null   object        
 3   Quantity           4384 non-null   int64         
 4   InvoiceDate        4384 non-null   datetime64[ns]
 5   Price              4384 non-null   float64       
 6   Customer ID        4383 non-null   float64       
 7   Country            4384 non-null   object        
 8   TransactionAmount  4384 non-null   float64       
 9   Recency            4384 non-null   int64         
 10  Frequency          4383 non-null   float64       
 11  MonetaryValue      4383 non-null   float64       
dtypes: datetime64[ns](1), float64(5), int64(2), object(4)
memory usage: 445.2+ KB


In [10]:
data.dropna(inplace=True)

#Calculating RFM Scores<br>
Now let’s calculate the recency, frequency, and monetary scores:

In [11]:
# Define scoring  : Higher score for lower recency (more recent),Higher score for higher frequency and monetary value
recency_scores = [5,4,3,2,1]
frequency_scores = [1,2,3,4,5]
monetary_scores = [1,2,3,4,5]

In [12]:
data['RecencyScore'] = pd.cut(data['Recency'], bins=5, labels=recency_scores)
data['FrequencyScore'] = pd.cut(data['Frequency'], bins=5, labels=frequency_scores)
data['MonetaryScore'] = pd.cut(data['MonetaryValue'], bins=5, labels=monetary_scores)

convert their datatype into integers to use these scores

In [13]:
# Convert RFM scores to numeric type
data['RecencyScore'] = data['RecencyScore'].astype(int)
data['FrequencyScore'] = data['FrequencyScore'].astype(int)
data['MonetaryScore'] = data['MonetaryScore'].astype(int)

#RFM Value Segmentation<br>

In [14]:
data.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TransactionAmount,Recency,Frequency,MonetaryValue,RecencyScore,FrequencyScore,MonetaryScore
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.40,5400,63.0,1187.08,1,1,1
12,489436,48173C,DOOR MAT BLACK FLOCK,10,2009-12-01 09:06:00,5.95,13078.0,United Kingdom,59.50,5400,475.0,16451.55,1,1,1
31,489437,22143,CHRISTMAS CRAFT HEART DECORATIONS,6,2009-12-01 09:08:00,2.10,15362.0,United Kingdom,12.60,5400,40.0,613.08,1,1,1
54,489438,21329,DINOSAURS WRITING SET,28,2009-12-01 09:24:00,0.98,18102.0,United Kingdom,27.44,5400,635.0,341776.73,1,1,5
71,489439,22065,CHRISTMAS PUDDING TRINKET POT,12,2009-12-01 09:28:00,1.45,12682.0,France,17.40,5400,517.0,11657.59,1,1,1


In [15]:
# Calculate RFM score by combining the individual scores
data['RFM_Score'] = data['RecencyScore'] + data['FrequencyScore'] + data['MonetaryScore']

# Create RFM segments based on the RFM score
segment_labels = ['Low-Value', 'Mid-Value', 'High-Value']

# Use duplicates='drop' to handle any duplicate bin edges
data['Value Segment'] = pd.qcut(data['RFM_Score'], q=4, labels=segment_labels, duplicates='drop')

In [16]:
# RFM Segment Distribution
segment_counts = data['Value Segment'].value_counts().reset_index()
segment_counts.columns = ['Value Segment', 'Count']

pastel_colors = px.colors.qualitative.Pastel

# Create the bar chart
fig_segment_dist = px.bar(segment_counts, x='Value Segment', y='Count',
                          color='Value Segment', color_discrete_sequence=pastel_colors,
                          title='RFM Value Segment Distribution')

# Update the layout
fig_segment_dist.update_layout(xaxis_title='RFM Value Segment',
                              yaxis_title='Count',
                              showlegend=False)

# Show the figure
fig_segment_dist.show()


/usr/local/lib/python3.10/dist-packages/plotly/express/_core.py:1971: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby(required_grouper, sort=False)  # skip one_group groupers


#RFM Customer Segments

In [ ]:
# Create a new column for RFM Customer Segments
data['RFM Customer Segments'] = ''

# Assign RFM segments based on the RFM score
data.loc[data['RFM_Score'] >= 9, 'RFM Customer Segments'] = 'Champions'
data.loc[(data['RFM_Score'] >= 6) & (data['RFM_Score'] < 9), 'RFM Customer Segments'] = 'Potential Loyalists'
data.loc[(data['RFM_Score'] >= 5) & (data['RFM_Score'] < 6), 'RFM Customer Segments'] = 'At Risk Customers'
data.loc[(data['RFM_Score'] >= 4) & (data['RFM_Score'] < 5), 'RFM Customer Segments'] = "Can't Lose"
data.loc[(data['RFM_Score'] >= 3) & (data['RFM_Score'] < 4), 'RFM Customer Segments'] = "Lost"

# Print the updated data with RFM segments
print(data[['Customer ID', 'RFM Customer Segments']])

        Customer ID RFM Customer Segments
0           13085.0                  Lost
12          13078.0                  Lost
31          15362.0                  Lost
54          18102.0   Potential Loyalists
71          12682.0                  Lost
...             ...                   ...
521212      18269.0   Potential Loyalists
522042      13270.0   Potential Loyalists
522621      12942.0   Potential Loyalists
522900      13369.0   Potential Loyalists
524732      15211.0   Potential Loyalists

[4383 rows x 2 columns]


#RFM Analysis

Now let’s have a look at the number of customers in all the segments:


In [ ]:
import plotly.colors

pastel_colors = plotly.colors.qualitative.Pastel

segment_counts = data['RFM Customer Segments'].value_counts()

# Create a bar chart to compare segment counts
fig = go.Figure(data=[go.Bar(x=segment_counts.index, y=segment_counts.values,
                            marker=dict(color=pastel_colors))])

# Set the color of the Champions segment as a different color
champions_color = 'rgb(158, 202, 225)'
fig.update_traces(marker_color=[champions_color if segment == 'Champions' else pastel_colors[i]
                                for i, segment in enumerate(segment_counts.index)],
                  marker_line_color='rgb(8, 48, 107)',
                  marker_line_width=1.5, opacity=0.6)

# Update the layout
fig.update_layout(title='Comparison of RFM Segments',
                  xaxis_title='RFM Segments',
                  yaxis_title='Number of Customers',
                  showlegend=False)

fig.show()

Now let’s have a look at the recency, frequency, and monetary scores of all the segments:

In [ ]:
# Calculate the average Recency, Frequency, and Monetary scores for each segment
segment_scores = data.groupby('RFM Customer Segments')[['RecencyScore', 'FrequencyScore', 'MonetaryScore']].mean().reset_index()

# Create a grouped bar chart to compare segment scores
fig = go.Figure()

# Add bars for Recency score
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['RecencyScore'],
    name='Recency Score',
    marker_color='rgb(158,202,225)'
))

# Add bars for Frequency score
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['FrequencyScore'],
    name='Frequency Score',
    marker_color='rgb(94,158,217)'
))

# Add bars for Monetary score
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['MonetaryScore'],
    name='Monetary Score',
    marker_color='rgb(32,102,148)'
))

# Update the layout
fig.update_layout(
    title='Comparison of RFM Segments based on Recency, Frequency, and Monetary Scores',
    xaxis_title='RFM Segments',
    yaxis_title='Score',
    barmode='group',
    showlegend=True
)

fig.show()